## **Presented By:** 
### - Azzouzi Ousssama Abderraouf
### - Mohamed Saadi

# ===============================================================

# 1- Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D


# 2- loading datasets

In [19]:
data_path = '011/text_data/'

ada_joy = data_path + 'ada_joy.csv'
assistance_info = data_path + 'assistance_info.csv'
gaze_positions = data_path + 'gaze_positions.csv'
input_info = data_path + 'input_info.csv'
joint_positions = data_path + 'joint_positions.csv'
myo_emg = data_path + 'myo_emg.csv'
myo_imu = data_path + 'myo_imu.csv'
myo_ori = data_path + 'myo_ori.csv'
pupil_cal_eye0 = data_path + 'pupil_cal_eye0.csv'
pupil_cal_eye1 = data_path + 'pupil_cal_eye1.csv'
pupil_eye0 = data_path + 'pupil_eye0.csv'
pupil_eye1 = data_path + 'pupil_eye1.csv'
robot_position = data_path + 'robot_position.csv'
world_cal_positions = data_path + 'world_cal_positions.csv'

ada_joy_df = pd.read_csv(ada_joy)
assistance_info_df = pd.read_csv(assistance_info)
gaze_positions_df = pd.read_csv(gaze_positions)
input_info_df = pd.read_csv(input_info)
joint_positions_df = pd.read_csv(joint_positions)
myo_emg_df = pd.read_csv(myo_emg)
myo_imu_df = pd.read_csv(myo_imu)
myo_ori_df = pd.read_csv(myo_ori)
pupil_cal_eye0_df = pd.read_csv(pupil_cal_eye0)
pupil_cal_eye1_df = pd.read_csv(pupil_cal_eye1)
pupil_eye0_df = pd.read_csv(pupil_eye0)
pupil_eye1_df = pd.read_csv(pupil_eye1)
robot_position_df = pd.read_csv(robot_position)
world_cal_positions_df = pd.read_csv(world_cal_positions)

ada_joy_df.head(), assistance_info_df.head(), gaze_positions_df.head(), input_info_df.head(), joint_positions_df.head(), myo_emg_df.head(), myo_imu_df.head(), myo_ori_df.head(), pupil_cal_eye0_df.head(), pupil_cal_eye1_df.head(), pupil_eye0_df.head(), pupil_eye1_df.head(), robot_position_df.head(), world_cal_positions_df.head()



(   timestamp  world_index  world_index_corrected    axes_x  axes_y  axes_z  \
 0   5.243769          147                    156 -0.000000    -0.0    -0.0   
 1   5.251749          147                    156 -0.000000    -0.0    -0.0   
 2   5.259806          148                    157 -0.000000    -0.0    -0.0   
 3   5.267750          148                    157  0.004807    -0.0    -0.0   
 4   5.275674          148                    157  0.026684    -0.0    -0.0   
 
    buttons_0  buttons_1  
 0          0          0  
 1          0          0  
 2          0          0  
 3          0          0  
 4          0          0  ,
    timestamp  exec_twist_x  exec_twist_y  exec_twist_z  exec_twist_r  \
 0   1.114619           0.0           0.0           0.0          -0.0   
 1   1.140355           0.0           0.0           0.0          -0.0   
 2   1.165112           0.0           0.0           0.0          -0.0   
 3   1.190987           0.0           0.0           0.0          -0.0

In [20]:
# print the column names of the dataframes
# print('ada_joy_df columns: ', ada_joy_df.columns)
# print('assistance_info_df columns: ', assistance_info_df.columns)
# print('gaze_positions_df columns: ', gaze_positions_df.columns)
# print('input_info_df columns: ', input_info_df.columns)
# print('joint_positions_df columns: ', joint_positions_df.columns)
# print('myo_emg_df columns: ', myo_emg_df.columns)
# print('myo_imu_df columns: ', myo_imu_df.columns)
# print('myo_ori_df columns: ', myo_ori_df.columns)
# print('pupil_cal_eye0_df columns: ', pupil_cal_eye0_df.columns)
# print('pupil_cal_eye1_df columns: ', pupil_cal_eye1_df.columns)
# print('pupil_eye0_df columns: ', pupil_eye0_df.columns)
# print('pupil_eye1_df columns: ', pupil_eye1_df.columns)
# print('robot_position_df columns: ', robot_position_df.columns)
# print('world_cal_positions_df columns: ', world_cal_positions_df.columns)


In [39]:
# Combine datasets based on timestamp (outer join to keep all timestamps)
combined_df = pd.merge(ada_joy_df, gaze_positions_df, on='timestamp', how='outer')
combined_df = pd.merge(combined_df, myo_emg_df, on='timestamp', how='outer')

# Handling missing values (example: fill with mean)
for col in combined_df.columns:
    try:
        combined_df[col] = combined_df[col].astype(float)
    except ValueError:
        print(f"Could not convert column {col} to float")

combined_df = combined_df.fillna(combined_df.select_dtypes(include=[np.number]).mean())
combined_df = combined_df.fillna(0)

#  calculate the mean of each numeric column and convert to double
mean = combined_df.select_dtypes(include=[np.number]).mean()

mean = mean.astype(float)

print(mean)  # use dtype instead of type()

# Check for missing values
# print(combined_df.isnull().sum())

combined_df

Could not convert column base_data to float
timestamp                    42.049635
world_index_x              1189.037440
world_index_corrected_x    1209.473008
axes_x                        0.078272
axes_y                        0.038775
axes_z                        0.006880
buttons_0                     0.021550
buttons_1                     0.000000
world_index_y              1251.389817
world_index_corrected_y    1273.513316
confidence                    0.958544
norm_pos_x                    0.490116
norm_pos_y                    0.561925
gaze_point_3d_x              -3.946174
gaze_point_3d_y              -8.276281
gaze_point_3d_z             104.274399
eye_center0_3d_x             19.892211
eye_center0_3d_y             14.464232
eye_center0_3d_z            -20.926180
gaze_normal0_x               -0.183672
gaze_normal0_y               -0.146846
gaze_normal0_z                0.962968
eye_center1_3d_x            -40.230051
eye_center1_3d_y             14.874633
eye_center1_3d_z    

,timestamp,world_index_x,world_index_corrected_x,axes_x,axes_y,axes_z,buttons_0,buttons_1,world_index_y,world_index_corrected_y,...,world_index_corrected,emg0,emg1,emg2,emg3,emg4,emg5,emg6,emg7,moving
0,5.243769,147.00000,156.000000,-0.000000,-0.000000,-0.00000,0.00000,0.0,1251.389817,1273.513316,...,1270.523539,112.491365,236.613674,349.202508,227.340904,156.18926,108.908446,81.94748,91.183582,0.0
1,5.251749,147.00000,156.000000,-0.000000,-0.000000,-0.00000,0.00000,0.0,1251.389817,1273.513316,...,1270.523539,112.491365,236.613674,349.202508,227.340904,156.18926,108.908446,81.94748,91.183582,0.0
2,5.259806,148.00000,157.000000,-0.000000,-0.000000,-0.00000,0.00000,0.0,1251.389817,1273.513316,...,1270.523539,112.491365,236.613674,349.202508,227.340904,156.18926,108.908446,81.94748,91.183582,0.0
3,5.267750,148.00000,157.000000,0.004807,-0.000000,-0.00000,0.00000,0.0,1251.389817,1273.513316,...,1270.523539,112.491365,236.613674,349.202508,227.340904,156.18926,108.908446,81.94748,91.183582,0.0
4,5.275674,148.00000,157.000000,0.026684,-0.000000,-0.00000,0.00000,0.0,1251.389817,1273.513316,...,1270.523539,112.491365,236.613674,349.202508,227.340904,156.18926,108.908446,81.94748,91.183582,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24136,84.664951,1189.03744,1209.473008,0.078272,0.038775,0.00688,0.02155,0.0,1251.389817,1273.513316,...,2539.000000,42.000000,144.000000,230.000000,110.000000,50.00000,46.000000,40.00000,31.000000,0.0
24137,84.687752,1189.03744,1209.473008,0.078272,0.038775,0.00688,0.02155,0.0,1251.389817,1273.513316,...,2540.000000,48.000000,142.000000,249.000000,116.000000,47.00000,46.000000,39.00000,33.000000,0.0
24138,84.701477,1189.03744,1209.473008,0.078272,0.038775,0.00688,0.02155,0.0,1251.389817,1273.513316,...,2540.000000,47.000000,132.000000,246.000000,119.000000,50.00000,46.000000,39.00000,33.000000,0.0
24139,84.723974,1189.03744,1209.473008,0.078272,0.038775,0.00688,0.02155,0.0,1251.389817,1273.513316,...,2541.000000,44.000000,133.000000,229.000000,113.000000,48.00000,45.000000,38.00000,31.000000,0.0
